# Introduction

    - This classifier is trained on a dataset which has images of humans and horses.
    - Key points about the classifier are as follows:
        - Uses Tensorflow & Keras
        - Uses ImageGenerator to read images and auto label them
        - Uses a fancy CNN architecture (sorry fancy because it has multiple CNN layers)
        
    - Nothing more fancy beyond this really :)

In [ ]:
# Run this on unix machine to download the zip file
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip

In [1]:
import tensorflow as tf
import numpy as np
import os

In [14]:
# load training data for horses and humans
horses_dir_name = os.listdir('./horse-or-human/horses')
humans_dir_name = os.listdir('./horse-or-human/humans')

In [18]:
# Create training ImageGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(rescale=1/255)

train_gen = train_generator.flow_from_directory(directory='./horse-or-human/',
                                    target_size=(300,300), 
                                    class_mode='binary', 
                                    batch_size=30)

Found 1027 images belonging to 2 classes.


In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

# define model
model = Sequential()
model.add(Conv2D(filters=100, kernel_size=(3,3), input_shape=(300,300,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=100, kernel_size=(3,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=100, kernel_size=(3,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit_generator(train_gen,                                                    
                    steps_per_epoch=10, 
                    epochs=1)

Instructions for updating:
Use tf.cast instead.
 2/35 [>.............................] - ETA: 5:55 - loss: 1.4570 - acc: 0.5667

KeyboardInterrupt: 